In [1]:
import numpy as np
import random
import pandas as pd
import os

from common import OUTPUTPATH
from models import linear_pcc
import data

/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# estimators here
linear_pcc.fit(data.x1, data.y)

In [3]:
num_importance = 100 # defult values show all variables
from sklearn.inspection import permutation_importance

def individual_importance_dataframe(est, X, y, scoring='neg_log_loss', num_importance=100000, n_repeats=100, seeds=None):
    """
    This algorithm use linear estimators and get the importance variables where p < 0.05
    To easy comparison, we set importance score: 1- p
    """
    r = permutation_importance(est, X, y, n_repeats=n_repeats, scoring=scoring, random_state=seeds)
    non_zero_indx = np.where(r.importances_mean > 0)
    indx = r.importances_mean[non_zero_indx].argsort()[::-1]
    important_variables = X.columns[non_zero_indx][indx]
    importance_scores = r.importances_mean[non_zero_indx][indx]
    coef = est.coef_[0][non_zero_indx][indx]
    df = pd.DataFrame({'variables': important_variables[:num_importance], 'coef': coef[:num_importance], 'importance': importance_scores[:num_importance]})
    return df

def linear_importance_dataframe(est, X, y, scoring = 'neg_log_loss', num_importance=100000, n_repeats=100, seeds=None):
    """
    Combine all morphologies plots together
    """
    data1 = pd.concat([X, y], axis=1)
    cnt = None
    for i in range(3):
        col_indx = -4 + i

        estimator, X, y = est[i], data1.iloc[:, :col_indx], data.y.iloc[:, i]
        temp_df = individual_importance_dataframe(est=estimator, X=X, y=y, scoring=scoring, num_importance=num_importance, n_repeats=n_repeats, seeds=seeds)
        if not cnt: 
            df = temp_df
            cnt = 1
        else:
            df = pd.concat([df, temp_df], axis=1)

    upper_columns = ['Sphere', 'Worm', 'Vesicle']
    lower_columns = ['variables', 'coef', 'importance']
    df.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
    return df

show all importance dataframe for Sphere, Worm, Vesicle

In [4]:
linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=100000, seeds=1000)

/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sk

Phase         Sphere                                Worm            \
Property   variables      coef    importance   variables      coef   
0         mw_tot_cre  0.000568  6.478597e+00  mw_tot_cre -0.000434   
1             dp_cre -0.046120  3.091151e+00  mv_tot_cre  0.024796   
2         mv_tot_cre -0.034102  2.686515e+00      dp_cre  0.027430   
3           apol_cna  0.338995  1.689558e+00  mw_tot_cna -0.000255   
4         mv_tot_cna  0.112893  1.088051e+00    apol_cna -0.060722   
5         mw_tot_cna -0.000677  5.488024e-01  mv_tot_cna  0.023598   
6             dp_cna  0.070488  4.030307e-01        conc  0.071982   
7            psa_cna -0.048058  3.688261e-01      mw_cna  0.004913   
8            psa_cre -0.292014  2.539096e-01     psa_cre  0.054039   
9               conc -0.148649  9.256409e-02     psa_cna  0.007714   
10            mw_cre  0.034571  2.028880e-02        temp  0.019352   
11            mw_cna -0.003825  1.365112e-02      mw_cre -0.023964   
12         clogp_cna  0.075862  5.492190e-03      dp_cna  0.004715   
13          apol_cre  0.023918  8.440298e-04      sphere -0.014460   
14           charged  0.049148  7.407380e-04   clogp_cna -0.008207   
15        mass_ratio  0.023096  1.328675e-04    apol_cre  0.002430   
16         vol_ratio  0.022865  1.294214e-04     charged -0.004470   
17         clogp_cre  0.022092  1.192713e-04   clogp_cre -0.004260   
18              salt -0.001060  7.319766e-07  mass_ratio -0.001365   
19            mv_cre  0.000562  4.620364e-07   vol_ratio -0.001258   
20               NaN       NaN           NaN      mv_cna  0.000892   
21               NaN       NaN           NaN        salt -0.000335   
22               NaN       NaN           NaN      mv_cre  0.000125   
23               NaN       NaN           NaN         NaN       NaN   
24               NaN       NaN           NaN         NaN       NaN   

Phase                      Vesicle                          
Property    importance   variables      coef    importance  
0         6.301531e+00  mw_tot_cre -0.000203  2.329613e+00  
1         1.595216e+00  mv_tot_cre  0.013010  1.140847e+00  
2         1.472010e+00      dp_cre  0.015465  1.115013e+00  
3         1.530948e-01  mw_tot_cna -0.000153  5.825421e-02  
4         1.323355e-01     psa_cna  0.009726  3.360638e-02  
5         7.744897e-02      mw_cna -0.003527  1.776039e-02  
6         4.393955e-02    apol_cna -0.011883  8.319730e-03  
7         3.258519e-02        temp  0.013655  7.317012e-03  
8         2.486725e-02        conc  0.010836  3.864777e-03  
9         1.873763e-02  mv_tot_cna  0.003911  3.705008e-03  
10        7.138453e-03     psa_cre  0.006610  1.726032e-03  
11        6.993450e-03      mw_cre -0.006682  5.970029e-04  
12        3.360774e-03      sphere -0.003742  3.581131e-04  
13        6.992048e-04      dp_cna -0.002591  3.373108e-04  
14        2.888871e-04          ph -0.002484  1.328515e-04  
15        1.080387e-04   clogp_cna -0.000891  2.594414e-05  
16        8.026421e-05   clogp_cre -0.001010  2.401301e-05  
17        4.832762e-05    apol_cre  0.000326  1.481325e-05  
18        6.060935e-06        worm -0.000460  6.437597e-06  
19        5.204889e-06  mass_ratio -0.000432  4.967163e-06  
20        3.133455e-06   vol_ratio -0.000414  4.587426e-06  
21        3.538457e-07     charged -0.000390  3.649103e-06  
22        2.621833e-07      mv_cna  0.000102  5.033406e-07  
23                 NaN        salt -0.000048  6.266830e-08  
24                 NaN      mv_cre  0.000022  5.359768e-08

In [6]:
top5_linear_dataframe = linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=5, seeds=1000)
top5_linear_dataframe.to_csv(os.path.join(OUTPUTPATH, 'Top5_linear_dataframe.csv'))
top5_linear_dataframe

/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/Users/yluu0028/opt/anaconda3/lib/python3.9/site-packages/sk

Phase         Sphere                             Worm                       \
Property   variables      coef importance   variables      coef importance   
0         mw_tot_cre  0.000568   6.478597  mw_tot_cre -0.000434   6.301531   
1             dp_cre -0.046120   3.091151  mv_tot_cre  0.024796   1.595216   
2         mv_tot_cre -0.034102   2.686515      dp_cre  0.027430   1.472010   
3           apol_cna  0.338995   1.689558  mw_tot_cna -0.000255   0.153095   
4         mv_tot_cna  0.112893   1.088051    apol_cna -0.060722   0.132336   

Phase        Vesicle                       
Property   variables      coef importance  
0         mw_tot_cre -0.000203   2.329613  
1         mv_tot_cre  0.013010   1.140847  
2             dp_cre  0.015465   1.115013  
3         mw_tot_cna -0.000153   0.058254  
4            psa_cna  0.009726   0.033606